# Aula10.Ex02 - A matriz Model

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [33]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
import random as rand

### Inicializando janela

In [34]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 720
largura = 1280
window = glfw.create_window(largura, altura, "Câmeras - Matriz Model", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base nas transformações Model, View e Projection.

In [35]:
vertex_code = """
        attribute vec3 position;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [36]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [37]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [38]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [39]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [40]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [41]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [42]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando cubos

Existem diferentes formas de modelar um cubo. Nós usaremos uma estratégia baseada no Quadrado com TRIANGLE_STRIP, conforme vimos na Aula04.Ex05. Assim, um quadrado é modelado usando dois triângulos e precisamos de apenas quatro vértices para isso (devido ao TRIANGLE_STRIP).

Nessa aula, nós concatenamos vértices de num_cubos cubos, em que num_cubos é uma variável.

In [43]:
def qtd_vertices_cilindro(sectors, stacks, vertices_base):
    base = vertices_base * 2
    lado = sectors * stacks * 6
    return lado + base

In [44]:
def qtd_vertices_esfera(sectors, stacks):
    return sectors * stacks * 6

In [45]:
# preparando espaço para n cubos (cada cubo tem 24 vertices), m pirâmides (cada pirâmide tem 16 vertices), p cilindros (número variável de vértices) e q esferas

sectors_cilindro = 30  # quantidade de sectors por cilindro (longitude) 
stacks_cilindro = 30   # quantidade de stacks por cilindro (latitude)
base_cilindro = 30     # quantidade de vértices componentes de cada base do cilindro

sectors_esfera = 30     # quantidade de sectors por esfera (longitude)
stacks_esfera = 30      # quantidade de stacks por esfera (latitude)

vertices_cilindro = qtd_vertices_cilindro(sectors_cilindro, stacks_cilindro, base_cilindro)    # número de vértices por cilindro
vertices_esfera = qtd_vertices_esfera(sectors_esfera, stacks_esfera)                           # número de vértices por esfera

num_cubos = 6                   # número de cubos
num_piramides = 2               # número de pirâmides 
num_cilindros = 1               # número de cilindros
num_esferas = 1                 # número de esferas 

In [46]:
def get_cubo():
    cubo = [
    # Face 1
    (-0.1, -0.1, +0.1),
    (+0.1, -0.1, +0.1),
    (-0.1, +0.1, +0.1),
    (+0.1, +0.1, +0.1),

    # Face 2
    (+0.1, -0.1, +0.1),
    (+0.1, -0.1, -0.1),         
    (+0.1, +0.1, +0.1),
    (+0.1, +0.1, -0.1),
    
    # Face 3
    (+0.1, -0.1, -0.1),
    (-0.1, -0.1, -0.1),            
    (+0.1, +0.1, -0.1),
    (-0.1, +0.1, -0.1),

    # Face 4
    (-0.1, -0.1, -0.1),
    (-0.1, -0.1, +0.1),         
    (-0.1, +0.1, -0.1),
    (-0.1, +0.1, +0.1),

    # Face 5
    (-0.1, -0.1, -0.1),
    (+0.1, -0.1, -0.1),         
    (-0.1, -0.1, +0.1),
    (+0.1, -0.1, +0.1),
    
    # Face 6
    (-0.1, +0.1, +0.1),
    (+0.1, +0.1, +0.1),           
    (-0.1, +0.1, -0.1),
    (+0.1, +0.1, -0.1)]
    
    return cubo

In [47]:
def get_piramide():
    piramide = [
        # base quadrada
        (-0.1,-0.1,+0.1),
        (+0.1,-0.1,+0.1),
        (-0.1,-0.1,-0.1),
        (+0.1,-0.1,-0.1), 

        # triângulo 1   
        (-0.1,-0.1,-0.1),
        (+0.1,-0.1,-0.1),
        (0,+0.1,0),    

        # triângulo 2                        
        (+0.1,-0.1,+0.1),
        (+0.1,-0.1,-0.1),
        (0,+0.1,0),

        # triângulo 3                             
        (-0.1,-0.1,-0.1),
        (-0.1,-0.1,+0.1),
        (0,+0.1,0),

        # triângulo 4                               
        (-0.1,-0.1,+0.1),
        (+0.1,-0.1,+0.1),
        (0,+0.1,0)]

    return piramide

In [48]:
# Entrada: ângulo e altura
# Saida: coordenadas no cilindro
def coordenadas_cilindro(t,h):
    x = math.cos(t)
    y = math.sin(t)
    z = h
    return (x,y,z)

def get_cilindro(sectors,stacks,base):
    PI = 3.141592
    sector_step = (PI*2)/sectors        # variar de 0 até 2π
    stack_step = 1.0/stacks             # variar de 0 até h
    vertices = []

    un = 0
    vn = 0

    for i in range(0,sectors):    

        u = i * sector_step                             # ângulo do sector                                              
        if i+1 != sectors: un = (i+1) * sector_step     # ângulo do sector da próxima iteração
        else: un = PI*2                                 # ângulo do sector definido na última iteração para não deixar lacunas no cilindro por causa da aproximação

        for j in range(0,stacks):   
        
            v = j * stack_step                              # altura da stack
            if j+1 != stacks: vn = (j+1) * stack_step       # altura da stack da próxima iteração
            else: vn = 1.0                                  # altura da stack definida na última iteração para não deixar lacunas no cilindro por causa da aproximação
        
            p0 = coordenadas_cilindro(u,v)
            p1 = coordenadas_cilindro(u,vn)
            p2 = coordenadas_cilindro(un,v)
            p3 = coordenadas_cilindro(un,vn)
        
            # triangulo 1 (primeira parte do poligono)
            vertices.append(p0)
            vertices.append(p2)
            vertices.append(p1)
        
            # triangulo 2 (segunda e ultima parte do poligono)
            vertices.append(p3)
            vertices.append(p1)
            vertices.append(p2)

    # tampas do cilindro
    base_sup = []
    base_inf = []
    angle = 0.0

    for i in range(base):
        angle += 2*PI / base 
        x = math.cos(angle)
        y = math.sin(angle)
        ci = (x,y,0.0)
        cs = (x,y,1.0)
        base_inf.append(ci)
        base_sup.append(cs)

    vertices.extend(base_inf)
    vertices.extend(base_sup)

    return vertices

In [49]:
# Entrada: ângulo de longitude e latitude
# Saida: coordenadas na esfera
def coordenadas_esfera(u,v):
    x = math.cos(u) * math.sin(v)
    y = math.sin(u) * math.sin(v)
    z = math.cos(v)
    return (x,y,z)

def get_esfera(sectors,stacks):
    PI = 3.141592
    sector_step = (2*PI)/sectors
    stack_step = PI/stacks
    vertices = []

    # ângulos pertecentes à iteração seguinte
    un = 0
    vn = 0

    # calcula a lista de vértices da esfera
    for i in np.arange(sectors):

        u = i * sector_step                          # ângulo do sector
        if i+1 != sectors: un = (i+1) * sector_step  # ângulo do sector da próxima iteração
        else: un = 2*PI                              # ângulo do sector definido na última iteração para não deixar lacunas na esfera por causa da aproximação

        for j in np.arange(stacks):

            v = j * stack_step                        # ângulo da stack
            if j+1 != stacks: vn = (j+1) * stack_step # ângulo da stack da próxima iteração
            else: vn = PI                             # ângulo da stack definido na última iteração para não deixar lacunas na esfera por causa da aproximação

            # cálculo dos vértices componentes da esfera
            p0 = coordenadas_esfera(u,v) 
            p1 = coordenadas_esfera(u,vn)
            p2 = coordenadas_esfera(un,v)
            p3 = coordenadas_esfera(un,vn)

            # primeira parte do polígono
            vertices.append(p0)
            vertices.append(p2)
            vertices.append(p1)
            #segunda parte do polígono
            vertices.append(p3)
            vertices.append(p1)
            vertices.append(p2)

    return vertices

In [50]:
# preenchendo o vetor de vertices com todas as figuras geométricas 
if num_cubos > 0: cubos = get_cubo()                                                             # cubo número 1
if num_piramides > 0: piramides = get_piramide()                                                 # pirâmide número 1
if num_cilindros > 0: cilindros = get_cilindro(sectors_cilindro,stacks_cilindro,base_cilindro)   # cilindro número 1
if num_esferas > 0: esferas = get_esfera(sectors_esfera,stacks_esfera)                           # esfera número 1

for i in range(1,num_cubos):                           # pegando o restante dos outros cubos
    vert_cubo = get_cubo()                             # pegando um novo cubo
    cubos = np.concatenate((cubos, vert_cubo), axis=0) # adicionando os vertices do cubo no nosso vetor de vertices

for i in range(1,num_piramides):                                        # pegando o restante das pirâmides
    vert_piramide = get_piramide()                                      # pegando uma nova pirâmide
    piramides = np.concatenate((piramides, vert_piramide), axis = 0)    # adicionando os vértices da pirâmide no nosso vetor de vértices

for i in range(1,num_cilindros):                                                    # pegando o restante dos cilindros
    vert_cilindro = get_cilindro(sectors_cilindro,stacks_cilindro,base_cilindro)    # pegando um novo cilindro
    cilindros = np.concatenate((cilindros, vert_cilindro), axis = 0)                # adicionando os vértices do cilindro no nosso vetor de vértices

for i in range(1,num_esferas):                                   # pegando o restante das esferas
    vert_esfera = get_esfera(sectors_esfera,stacks_esfera)       # pegando uma nova esfera
    esferas = np.concatenate((esfera, vert_esfera), axis = 0)    # adicionando os vértices da esfera no nosso vetor de vértices

size = num_cubos*24 + num_piramides*16 + num_cilindros*vertices_cilindro + num_esferas*vertices_esfera
shapes = []

if num_cubos > 0: shapes.extend(cubos)
if num_piramides > 0: shapes.extend(piramides)
if num_cilindros > 0: shapes.extend(cilindros)
if num_esferas > 0: shapes.extend(esferas)

vertices = np.zeros(size, [("position", np.float32, 3)])
vertices['position'] = np.array(shapes)

In [51]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [52]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [53]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [54]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [55]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

###  Vamos pegar a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela!

In [56]:
loc_color = glGetUniformLocation(program, "color")

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [57]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);


def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp
    
    cameraSpeed = 0.01
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)
    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [58]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)

    matrix_transform = glm.mat4(1.0)                                                # instanciando uma matriz identidade   
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z)) # aplicando rotacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))     # aplicando translacao 
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))         # aplicando escala
    matrix_transform = np.array(matrix_transform).T                                 # pegando a transposta da matriz (glm trabalha com ela invertida)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 100.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [59]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.


Usaremos o GL_TRIANGLE_STRIP e modelaremos uma face do Cubo por vez, por questões didáticas. Iremos colorir cada face do Cubo com uma cor diferente.

In [60]:
def desenha_cubo(num_cubo, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    cores_face = [
        [0.92, 0.92, 0.92], # R, G, B
        [0.88, 0.88, 0.88],
        [0.84, 0.84, 0.84],
        [0.80, 0.80, 0.80],
        [0.76, 0.76, 0.76],        
        [0.72, 0.72, 0.72],
    ]
    
    # DESENHANDO O CUBO
    face = 0
    for i in range(num_cubo*24,(num_cubo+1)*24,4): # incremento de 4 em 4 (desenhando cada face)
        R = cores_face[face][0]
        G = cores_face[face][1]
        B = cores_face[face][2]
        glUniform4f(loc_color, R, G, B, 1.0) ### definindo uma cor
        glDrawArrays(GL_TRIANGLE_STRIP, i, 4) ## renderizando
        face+=1

    return

In [61]:
def desenha_piramide(num_piramide, angle, r, t, z, offset):

    mat_model = model(angle, r[0], r[1], r[2], t[0], t[1], t[2], s[0], s[1], s[2])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    # número de vértices da base da pirâmide
    vertices_base = 4

    # lista de cores aplicada em cada uma das faces 
    cores_face = [
        # R, G, B
        [0.85, 0.85, 0.85], 
        [0.82, 0.82, 0.82],
        [0.76, 0.76, 0.76],
        [0.73, 0.73, 0.73],
        [0.70, 0.70, 0.70],
    ]

    # DESENHANDO A PIRÂMIDE 

    # cores da base
    R = cores_face[0][0]
    G = cores_face[0][1]
    B = cores_face[0][2]
    # base
    i = num_piramide*16 + offset
    glUniform4f(loc_color, R, G, B, 1.0)   
    glDrawArrays(GL_TRIANGLE_STRIP, i, 4)

    # faces
    triangulo = 0
    for i in range(num_piramide*16 + vertices_base + offset,(num_piramide + 1)*16 + offset,3):
        R = cores_face[triangulo + 1][0]
        G = cores_face[triangulo + 1][1]
        B = cores_face[triangulo + 1][2]

        glUniform4f(loc_color, R, G, B, 1.0)  # definindo uma cor
        glDrawArrays(GL_TRIANGLE_STRIP, i, 3) # renderizando
        triangulo += 1

    return

In [62]:
def desenha_cilindro(num_cilindro, qtd_vertices, qtd_base, angle, r, t, z, offset):

    mat_model = model(angle, r[0], r[1], r[2], t[0], t[1], t[2], s[0], s[1], s[2])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    for triangle in range(num_cilindro*qtd_vertices + offset,(num_cilindro + 1)*qtd_vertices + offset - 2*qtd_base,3):
       rand.seed(triangle)
       color = rand.randint(70,91)
       color /= 100        
       glUniform4f(loc_color, color, color, color, 1.0)
       
       glDrawArrays(GL_TRIANGLES, triangle, 3)  

    offset_lado = (num_cilindro * qtd_vertices) + (qtd_vertices - 2 * qtd_base) + offset

    #tampa inferior
    color = rand.randint(70,91)
    color /= 100
    glUniform4f(loc_color, color, color, color, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset_lado, qtd_base) 

    #tampa superior
    color = rand.randint(70,91)
    color /= 100
    glUniform4f(loc_color, color, color, color, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset_lado + qtd_base,qtd_base) 

    return

In [63]:
def desenha_esfera(num_piramide, qtd_vertices, angle, r, t, z, offset):

    mat_model = model(angle, r[0], r[1], r[2], t[0], t[1], t[2], s[0], s[1], s[2])
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    for triangulo in range(num_esfera*qtd_vertices + offset,(num_esfera + 1)*qtd_vertices + offset,3):
        rand.seed(triangulo)
        color = rand.randint(70,91)
        color /= 100

        glUniform4f(loc_color, color, color, color, 1.0)  # definindo uma cor
        glDrawArrays(GL_TRIANGLES, triangulo, 3)          # renderizando

    return

In [64]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

angulo_obj = 0
translacao_qtd = 0.4
translacao_lado = False
        
while not glfw.window_should_close(window):

    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    # computando e enviando matrizes Model, View e Projection para a GPU
    
    # temos uma matriz model por objeto!
    num_cubo=0   
    # angulo de rotacao e eixos
    angle=0.0; r_x=0.0; r_y=0.0; r_z=1.0
    # translacao
    t_x=0.4; t_y=0.08; t_z=-0.12
    # escala
    s_x=0.2; s_y=1.5; s_z=0.2
    desenha_cubo(num_cubo, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    
    
    num_cubo=1
    # angulo de rotacao e eixos
    angle=0.0; r_x=0.0; r_y=0.0; r_z=1.0
    # translacao
    t_x=-0.4; t_y=0.08; t_z=-0.12
    # escala
    s_x=0.2; s_y=1.5; s_z=0.2
    desenha_cubo(num_cubo, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    
    
    # temos uma matriz model por objeto!
    num_cubo=2  
    # angulo de rotacao e eixos
    angle=0.0; r_x=0.0; r_y=0.0; r_z=1.0
    # translacao
    t_x=0.4; t_y=0.08; t_z=0.12
    # escala
    s_x=0.2; s_y=1.5; s_z=0.2
    desenha_cubo(num_cubo, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    
    
    num_cubo=3
    # angulo de rotacao e eixos
    angle=0.0; r_x=0.0; r_y=0.0; r_z=1.0
    # translacao
    t_x=-0.4; t_y=0.08; t_z=0.12
    # escala
    s_x=0.2; s_y=1.5; s_z=0.2
    desenha_cubo(num_cubo, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    
    
    num_cubo=4
    # angulo de rotacao e eixos
    angle=180.0; r_x=1.0; r_y=0.0; r_z=0.0
    # translacao
    t_x=0.0; t_y=0.2; t_z=0.0
    # escala
    s_x=15.0; s_y=0.1; s_z=15.0
    desenha_cubo(num_cubo, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)


    num_cubo=5
    # angulo de rotacao e eixos
    angle=0.0; r_x=0.0; r_y=0.0; r_z=1.0
    # translacao
    t_x=0.0; t_y=0.25; t_z=0.0
    # escala
    s_x=5.0; s_y=0.2; s_z=2.0
    desenha_cubo(num_cubo, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)

    # espaço ocupado pelos vértices dos cubos
    offset = num_cubos * 24

    # PIRÂMIDE 0
    num_piramide = 0
    angle = angulo_obj             # ângulo de rotação
    r = [0.0, 1.0, 0.0]            # eixos de rotação
    t = [0.0, 0.6, 0.0]            # translação
    s = [1.0, 1.0, 1.0]            # escala
    desenha_piramide(num_piramide, angle, r, t, s, offset)

    angulo_obj += 1
    if angulo_obj == 360: 
        angulo_obj = 0

    # PIRÂMIDE 1
    num_piramide = 1
    angle = 30.0            # ângulo de rotação
    r = [0.0, 0.0, 1.0]     # eixos de rotação
    t = [0.5, 0.3, 0.6]     # translação
    s = [0.5, 0.8, 0.5]     # escala
    desenha_piramide(num_piramide, angle, r, t, s, offset)

    offset = (24 * num_cubos) + (16 * num_piramides)

    if translacao_qtd <= 0.2: translacao_lado = False
    if translacao_qtd >= 0.4: translacao_lado = True

    if not translacao_lado: translacao_qtd += 0.01
    else: translacao_qtd -= 0.01

    # CILINDRO 0
    num_cilindro = 0
    angle = 180.0                        # ângulo de rotação
    r = [0.0, 1.0, 1.0]                  # eixos de rotação
    t = [-translacao_qtd, 1.0, 0.2]      # translação
    s = [0.02, 0.02, 0.09]               # escala
    desenha_cilindro(num_cilindro, vertices_cilindro, base_cilindro, angle, r, t, s, offset)

    offset = (24 * num_cubos) + (16 * num_piramides) + (vertices_cilindro * num_cilindros) 

    # ESFERA 0  
    num_esfera = 0
    angle = angulo_obj        # ângulo de rotação
    r = [0.0, 1.0, 1.0]       # eixos de rotação
    t = [0.6, 0.3, 0.4]       # translação
    s = [0.03, 0.03, 0.03]    # escala
    desenha_esfera(num_esfera, vertices_esfera, angle, r, t, s, offset)

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    
     
    glfw.swap_buffers(window)

glfw.terminate()

# Exercício

Modifique esse código para incluir outros objetos 3D na sua cena, como esferas e pirâmides.
* Cada objeto deve ser iniciado com seu centro próximo da coordenada (0,0,0) no Espaço Local.
* Cada objeto deve ter sua própria matriz Model, que o posiciona em relação ao Espaço Mundo.